In [ ]:
! conda install conda-forge 

In [74]:
# http://pordlabs.ucsd.edu/cjiang/python.html

from scipy.io import netcdf
import pandas as pd
import numpy as np

f = netcdf.netcdf_file('uv3z.nc', 'r')

In [7]:
f.variables

OrderedDict([('water_u', <scipy.io.netcdf.netcdf_variable at 0x10d01cef0>),
             ('time', <scipy.io.netcdf.netcdf_variable at 0x10d01ceb8>),
             ('depth', <scipy.io.netcdf.netcdf_variable at 0x10d027048>),
             ('lat', <scipy.io.netcdf.netcdf_variable at 0x10d027128>),
             ('lon', <scipy.io.netcdf.netcdf_variable at 0x10d027208>),
             ('water_v', <scipy.io.netcdf.netcdf_variable at 0x10d027320>)])

In [232]:
df = pd.read_csv('large_sample.csv')

In [233]:
df.describe()

,Unnamed: 0,time,lat,lon,u,v
count,616416.000000,616416.000000,616416.000000,616416.000000,616416.000000,616416.000000
mean,353905.727872,157428.000000,38.675463,145.797065,0.206454,-0.041989
std,204038.252223,221.594403,2.013076,5.713493,0.389184,0.341566
min,45.000000,157056.000000,35.040001,134.959961,-1.348000,-2.318000
25%,176737.500000,157242.000000,36.959999,141.839966,-0.026000,-0.235000
50%,353430.000000,157428.000000,38.720001,146.239990,0.143000,-0.049000
75%,530122.500000,157614.000000,40.400002,150.640015,0.368000,0.146000
max,706815.000000,157800.000000,42.000000,154.959961,2.737000,1.645000


In [234]:
df.head()

,Unnamed: 0,time,lat,lon,u,v
0,45,157056.0,35.040001,138.560059,-0.009,-0.034
1,46,157056.0,35.040001,138.640015,-0.014,-0.016
2,47,157056.0,35.040001,138.719971,-0.005,0.041
3,53,157056.0,35.040001,139.199951,0.047,-0.152
4,54,157056.0,35.040001,139.280029,0.209,0.288


In [235]:
df['magnitude'] = np.sqrt(np.square(df['u']) + np.square(df['v']))

In [236]:
def get_angle(row):
    if row['u'] < 0 and row['v'] > 0:
        angle = 180*np.arctan(np.abs(row['u']/row['v']))/np.pi + 90
    elif row['u'] < 0 and row['v'] < 0:
        angle = 180*np.arctan(np.abs(row['u']/row['v']))/np.pi + 180
    elif row['u'] > 0 and row['v'] < 0:
        angle = 360 + 180*np.arctan(row['u']/row['v'])/np.pi
    else:
        angle = 180*np.arctan(np.abs(row['u']/row['v']))/np.pi
    return angle

In [238]:
df['angle'] = df.apply(get_angle, axis=1)

/Users/michelleho/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in double_scalars
  if __name__ == '__main__':
/Users/michelleho/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


In [ ]:
df['angle'].describe()

In [ ]:
np.arctan(1/)

In [91]:
df['angle'].describe()

count    16748.000000
mean        -0.082583
std          0.995506
min         -1.570796
25%         -1.000424
50%         -0.156462
75%          0.890688
max          1.570796
Name: angle, dtype: float64

In [251]:
df['date'] = pd.to_datetime('2000-01-01') + pd.to_timedelta(df['time'], unit='h')

In [255]:
df.dtypes

Unnamed: 0             int64
time                 float64
lat                  float64
lon                  float64
u                    float64
v                    float64
magnitude            float64
angle                float64
date          datetime64[ns]
dtype: object

In [252]:
df.to_csv('large_sample_processed.csv')

In [219]:
# Code for animation with ipywidgets (needs to be adapted!)

In [ ]:
import matplotlib as mpl
import matplotlib.cm as cm

plt.figure(figsize=(8,10))
jet = plt.get_cmap('rainbow')

for key, value in df_vectors.items():
    soa = np.array(value)
    X, Y, U, V = zip(*soa)
    N = np.sqrt(np.array(U)**2+np.array(V)**2)
    U2, V2 = (U/N)/400, (V/N)/400
    ax = plt.gca()
    ax.quiver(Y, X, U2, V2, angles='xy', scale_units='xy', scale=1,color=jet(float(key)/23))
    ax.set_ylim([stations['latitude'].min()-.01, stations['latitude'].max()+.01])
    ax.set_xlim([stations['longitude'].min()-.01, stations['longitude'].max()+.01])
    ax.scatter(stations['longitude'],stations['latitude'],c='k',s=2)

plt.draw()
plt.show()


In [87]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


def plotByHour(hour):
    plt.figure(figsize=(8,10))
    soa = np.array(df_vectors[hour])
    X, Y, U, V = zip(*soa)
    N = np.sqrt(np.array(U)**2+np.array(V)**2)
    U2, V2 = (U/N)/400, (V/N)/400
    ax = plt.gca()
    ax.quiver(Y, X, U2, V2, angles='xy', scale_units='xy', scale=1, color=jet(float(hour)/23))
    ax.set_ylim([stations['latitude'].min()-.01, stations['latitude'].max()+.01])
    ax.set_xlim([stations['longitude'].min()-.01, stations['longitude'].max()+.01])
    ax.scatter(stations['longitude'],stations['latitude'], s=2, c='k')
    plt.draw()
    return plt.show()


In [89]:
df_vectors

NameError: name 'df_vectors' is not defined

In [88]:
interact(plotByHour, hour=widgets.IntSlider(min=0,max=2,step=1,value=0))

<function __main__.plotByHour>